In [99]:
import gzip
import pickle
from collections import deque
import numpy as np
import time

import import_ipynb
from simulation import simul

In [110]:
# update RumorSpreadAbility
def update_rsa(adj_mat, target_edge, rsa):
    n = len(adj_mat)
    visit = [False]*n
    rsa_new = np.copy(rsa)
    
    s,t = target_edge
    rsa_new[s] -= adj_mat[s,t]*rsa[t]
    visit[s] = True

    Q = deque([s])
    while Q:
        u = Q.popleft()
        for v in np.where(adj_mat[:,u]>0)[0]:
            rsa_new[v] -= adj_mat[v,u]*(rsa[u]-rsa_new[u])
            if not visit[v]:
                Q.append(v)
                visit[v] = True

    return rsa_new


def update_rsa_original(adj_mat, target_edge, rsa):
    n = len(adj_mat)
    visit = [False]*n
    rsa_new = np.copy(rsa)
    
    s,t = target_edge
    rsa_new[s] -= adj_mat[s,t]*rsa[t]
    visit[s] = True

    Q = deque([s])
    while Q:
        u = Q.popleft()
        for v in np.where(adj_mat[:,u]>0)[0]:
            if visit[v]: continue
            for w in np.where(adj_mat[v,:]>0)[0]: rsa_new[v] -= adj_mat[v,w]*rsa[w]
            Q.append(v)
            visit[v] = True

    return rsa_new
    


def update_prob(adj_mat, target_edge, prob):
    n = len(adj_mat)
    visit = [False]*n
    prob_delta = [0]*n

    s,t = target_edge
    prob_delta[t] = adj_mat[s,t]*prob[s]*(1-prob[t])/(1-adj_mat[s,t]*prob[s])
    visit[t]=True
    
    Q = deque([t])
    while Q:
        u = Q.popleft()
        for v in np.where(adj_mat[u,:]>0)[0]:
            prob_delta[v] += (1-prob[v])*adj_mat[u,v]*prob_delta[u]/(1-adj_mat[u,v]*prob[u])
            if not visit[v]:
                Q.append(v)
                visit[v] = True

    return [prob[i]-prob_delta[i] for i in range(n)]

In [131]:
def MDS(adj_list, seed_idx, del_edge_num, use_orig=False):
    time_start = time.time()
    time_alg = []
    mask = []
    
    # convert adj list to adj matrix & make edge list
    n = len(adj_list)
    adj_mat = [[0]*n for _ in range(n)]
    edge = []
    for u in range(n):
        for v,p in adj_list[u]:
            adj_mat[u][v] = p
            edge.append([u,v,p])
    adj_mat = np.array(adj_mat)

    # compute initial probability
    prob = simul(adj_list, seed_idx)

    # compute initial RSA
    d = 6  # diameter. but use the same value as the original paper
    rsa = np.array([1.0]*n)
    tmp = np.array([1.0]*n)
    for i in range(1,d+1):
        tmp = adj_mat@tmp
        rsa += tmp

    # select edges
    for _ in range(del_edge_num):
        max_score = 0
        max_i = 0
        for i,(v,u,p) in enumerate(edge):
            score = (1-prob[u])*(rsa[v]+rsa[u])
            if max_score<score: max_score,max_i=score,i 
        target_edge = edge[max_i][:2]

        rsa = update_rsa(adj_mat, target_edge, rsa) if not use_orig else update_rsa_original(adj_mat, target_edge, rsa)
        prob = update_prob(adj_mat, target_edge, prob)
        
        mask.append(edge[max_i])
        del edge[max_i]
        adj_mat[target_edge[0],target_edge[1]]=0
        
        time_alg.append(time.time()-time_start)

    return mask, time_alg